<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/Reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anytree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 1.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [4]:
import pickle

# Reading from a file using Pickle
with open(f"{base_path}conversations_with_sentiment_all.pkl", 'rb') as file:
    conversations = pickle.load(file)

# Print the loaded variables
print("Len conversations:", len(conversations))

Len conversations: 18184


In [5]:
import spacy

nlp = spacy.load("en_core_web_sm")

def contains_reasoning(text):
    doc = nlp(text)

    # Check for specific part-of-speech patterns indicating reasoning
    reasoning_patterns = ["because", "since", "therefore", "due to", "as a result", "consequently", "thus",
                          "for this reason", "in conclusion", "owing to", "on account of", "resulting in",
                          "so", "hence", "in light of", "accordingly", "on the grounds that"]
    #based on,

    for token in doc:
        if token.text.lower() in reasoning_patterns:
            return True

    return False

In [6]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="typeform/distilbert-base-uncased-mnli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/258 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [7]:
VERBOSE = 3000

In [8]:
import pickle

def write_conversations_with_reasoning():
  with open(f"{base_path}conversations_reasoning_with_one_shot.pkl", 'wb') as file:
    pickle.dump(conversations, file)
  print("Variables written to the file.")

In [9]:
def extract_comments(conversations, num_break_points):
  flat_comments_list = []
  break_points_index = []
  counter = 0
  break_length = int(len(conversations)/num_break_points)
  print("break len ", break_length)
  for id in conversations:
    if counter % break_length == 0:
      break_points_index.append(len(flat_comments_list))
    counter += 1
    comments = conversations[id]['comments']
    for comment in comments:
      flat_comments_list.append(comment)

  return flat_comments_list, break_points_index

In [10]:
import pickle

def write_reasoning_dic(reasoning_dic, file_path):
  with open(file_path, 'wb') as file:
      pickle.dump(reasoning_dic, file)
  print("Variables written to the file.")

In [11]:
import pickle

def merge_conversations_with_reasoning(conversations, file_pathes, method):
  reasoning_dic = dict()
  for file_path in file_pathes:
    with open(file_path, 'rb') as file:
      reasoning_part = pickle.load(file)
      reasoning_dic.update(reasoning_part)

  for id in conversations:
    comments = conversations[id]['comments']
    for comment in comments:
      if not(comment['body'] == '[deleted]' or comment['body'] == '[removed]'):
        if method != "flasher":
          comment['has_reasoning'+method] = reasoning_dic[comment['id']]
        else:
          comment['has_reasoning'] = reasoning_dic[comment['id']]
  #watch out for this path, rewrite carefully
  with open(f"{base_path}conversations_with_one_shot_reasoning.pkl", 'wb') as file:
    pickle.dump(conversations, file)
  print("Variables written to the file.")

In [12]:
from transformers import pipeline

def reasoning_based_on_one_shot_classification(texts):
    scores = []
    labels = ['justification', "no justification"]
    hypothesis_template = "This example uses {}."

    for text in texts:
        if not text.strip():  # Check if the text is empty or contains only whitespace
            scores.append(-1)
            continue
        result = classifier(text, labels, hypothesis_template=hypothesis_template, multi_label=False)
        for label, score in zip(result["labels"], result["scores"]):
            if label == "justification":
                scores.append(score)
    return scores


In [13]:
def calculate_reasoning_for_corpus(corpus, method):
  reasoning_dic = dict()
  comments_text = []
  comments_id_dic = dict()
  for comment in corpus:
      if not(comment['body'] == '[deleted]' or comment['body'] == '[removed]'):
          if method == "flasher":
            has_reasoning = contains_reasoning(comment['body'])
            reasoning_dic[comment['id']] = has_reasoning
          elif method == "one-shot":
            comments_text.append(comment['body'])
            comments_id_dic[comment['id']] = len(comments_text) - 1
  if method == "one-shot":
    scores = reasoning_based_on_one_shot_classification(comments_text)
    for comment in corpus:
      if not(comment['body'] == '[deleted]' or comment['body'] == '[removed]'):
        reasoning_dic[comment['id']] = scores[comments_id_dic[comment['id']]]
  return reasoning_dic

In [14]:
import os

def calculate_reasoning(conversations, num_of_parts, rewrite, method):
  flat_comments, break_points_index = extract_comments(conversations, num_of_parts)
  print("len ", len(break_points_index))
  break_points_index.append(len(flat_comments))
  file_pathes = []
  for i in range(1, len(break_points_index)):
    print("range ", break_points_index[i-1], break_points_index[i])
    file_path = f"{base_path}comments_reasoning_{break_points_index[i-1]}_{break_points_index[i]}.pkl"
    if method != "flasher":
      file_path = f"{base_path}comments_{method}_reasoning_{break_points_index[i-1]}_{break_points_index[i]}.pkl"
    file_pathes.append(file_path)
    print("path ", file_path)
    if rewrite or not os.path.exists(file_path):
      print("do")
      corpus = flat_comments[break_points_index[i-1]:break_points_index[i]]
      reasoning_dic = calculate_reasoning_for_corpus(corpus, method)
      write_reasoning_dic(reasoning_dic, file_path)
  merge_conversations_with_reasoning(conversations, file_pathes, method)

In [15]:
calculate_reasoning(conversations, 100, False, "one-shot")
write_conversations_with_reasoning()

break len  181
len  101
range  0 3333
path  /content/drive/MyDrive/University/RedditData/comments_one-shot_reasoning_0_3333.pkl
range  3333 5578
path  /content/drive/MyDrive/University/RedditData/comments_one-shot_reasoning_3333_5578.pkl
range  5578 8535
path  /content/drive/MyDrive/University/RedditData/comments_one-shot_reasoning_5578_8535.pkl
range  8535 11989
path  /content/drive/MyDrive/University/RedditData/comments_one-shot_reasoning_8535_11989.pkl
range  11989 14440
path  /content/drive/MyDrive/University/RedditData/comments_one-shot_reasoning_11989_14440.pkl
range  14440 17860
path  /content/drive/MyDrive/University/RedditData/comments_one-shot_reasoning_14440_17860.pkl
range  17860 20026
path  /content/drive/MyDrive/University/RedditData/comments_one-shot_reasoning_17860_20026.pkl
range  20026 22345
path  /content/drive/MyDrive/University/RedditData/comments_one-shot_reasoning_20026_22345.pkl
range  22345 24109
path  /content/drive/MyDrive/University/RedditData/comments_one-s

In [ ]:
calculate_reasoning(conversations, 10, False, "flasher")
write_conversations_with_reasoning()

break len  1818
len  11
range  0 26053
path  /content/drive/MyDrive/University/RedditData/comments_reasoning_0_26053.pkl
do


KeyboardInterrupt: 